In [1]:
import pandas as pd
import numpy as np
import beam_search_module as bsm 
import ast
from plot_sg_and_pop import *
import json

In [2]:
def convert_timestamps(timestamp_str):
    # Remove the square brackets and split the string
    timestamps = timestamp_str.strip('[]').split(', ')
    # Extract the actual datetime strings and convert to datetime objects
    return [pd.Timestamp(ts.split('(')[-1].split(')')[0]) for ts in timestamps]


In [3]:
stock_df_all = pd.read_csv('data/stock_data_all_features.csv')
stock_df_all['growth_target'] = stock_df_all['growth_target'].apply(ast.literal_eval)
stock_df_all['target'] = stock_df_all['target'].apply(ast.literal_eval)
stock_df_all['target'] = stock_df_all['target'].apply(lambda x: [((i / x[0])-1)*100 for i in x])
stock_df = stock_df_all.copy()
stock_df = stock_df.drop(['target', 'dates', 'symbol'], axis=1)
stock_df_backup = stock_df.copy()
dates = convert_timestamps(stock_df_all.loc[0, 'dates'])
stock_df_all


,symbol,country,industry,currency,exchangeTimezoneName,exchange,sector,averageVolume10days,enterpriseToEbitda,marketCap,...,growth_best_day_of_week,growth_best_month,growth_best_quarter,growth_best_day_of_month,growth_best_week_of_month,growth_biggest_continuous_increase_perc,growth_biggest_continuous_decrease_perc,growth_biggest_continuous_increase_perc_start,growth_biggest_continuous_decrease_perc_start,growth_target
0,AOF.DE,Germany,Software—Application,EUR,Europe/Berlin,GER,Technology,8329.0,33.294,1.241838e+09,...,3,8,3,25,34,16.80891,-17.72197,93.0,287.0,"[0.0, 1.02, 0.1, -1.31, 2.49, -0.47, -0.47, 3...."
1,LFG.MI,Italy,Entertainment,EUR,Europe/Rome,MIL,Communication Services,330.0,-21.050,2.849466e+07,...,1,1,1,31,52,18.51535,-8.00000,64.0,69.0,"[0.0, -2.03, 0.0, 2.07, -2.03, 1.72, 1.69, -2...."
2,ENV.MI,Italy,Packaged Foods,EUR,Europe/Rome,MIL,Consumer Defensive,3831.0,8.794,5.971590e+07,...,2,9,3,28,37,11.23512,-6.57895,44.0,22.0,"[0.0, -3.33, -0.69, 4.17, -0.67, -0.67, -0.68,..."
3,FPO.F,United Kingdom,Other Precious Metals & Mining,EUR,Europe/Berlin,FRA,Basic Materials,108.0,1.402,2.123667e+08,...,0,1,4,3,40,25.00001,-21.52061,76.0,108.0,"[0.0, -0.13, -2.28, -1.17, -6.44, -1.54, -1.57..."
4,M07.F,United States,Security & Protection Services,EUR,Europe/Berlin,FRA,Industrials,0.0,14.054,5.451780e+09,...,4,4,2,14,15,9.97164,-10.17502,21.0,260.0,"[0.0, -0.67, -0.68, 1.36, 0.0, -0.67, -0.68, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982,INRN.SW,Switzerland,Specialty Industrial Machinery,CHF,Europe/Zurich,EBS,Industrials,1124.0,15.530,1.979479e+09,...,4,4,2,21,13,17.59738,-12.93413,59.0,259.0,"[0.0, -3.21, 0.83, -0.41, 0.62, -0.72, -0.72, ..."
7983,RICO.L,Japan,Business Equipment & Supplies,JPY,Europe/London,LSE,Industrials,1940.0,5.531,5.086319e+09,...,4,9,3,9,39,13.67696,-9.63956,290.0,64.0,"[0.0, 0.48, 0.0, -4.75, 0.0, 0.0, 0.0, 2.97, 0..."
7984,KNM.L,Japan,Electronic Gaming & Multimedia,JPY,Europe/London,LSE,Communication Services,3530.0,7.976,6.297095e+09,...,6,9,3,23,39,9.76675,-8.57953,197.0,48.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.51, 0.0..."
7985,MAW.L,Japan,Electronic Components,JPY,Europe/London,LSE,Technology,350.0,7.200,1.774490e+09,...,4,9,3,25,39,14.62789,-10.78806,65.0,137.0,"[0.0, 0.0, 0.0, 0.0, 2.95, 0.0, 0.0, 0.0, 0.0,..."


In [4]:
domain_attributes = True
time_series_attributes = True

if not domain_attributes and time_series_attributes:
    stock_df = stock_df.iloc[:, 11:]

if domain_attributes and not time_series_attributes:
    domain_features = stock_df.columns[:11]
    columns_keep = list(domain_features) + ['growth_target']
    stock_df = stock_df[columns_keep]

if domain_attributes and time_series_attributes:
    stock_df = stock_df
stock_df

,country,industry,currency,exchangeTimezoneName,exchange,sector,averageVolume10days,enterpriseToEbitda,marketCap,debtToEquity,...,growth_best_day_of_week,growth_best_month,growth_best_quarter,growth_best_day_of_month,growth_best_week_of_month,growth_biggest_continuous_increase_perc,growth_biggest_continuous_decrease_perc,growth_biggest_continuous_increase_perc_start,growth_biggest_continuous_decrease_perc_start,growth_target
0,Germany,Software—Application,EUR,Europe/Berlin,GER,Technology,8329.0,33.294,1.241838e+09,32.492,...,3,8,3,25,34,16.80891,-17.72197,93.0,287.0,"[0.0, 1.02, 0.1, -1.31, 2.49, -0.47, -0.47, 3...."
1,Italy,Entertainment,EUR,Europe/Rome,MIL,Communication Services,330.0,-21.050,2.849466e+07,181.181,...,1,1,1,31,52,18.51535,-8.00000,64.0,69.0,"[0.0, -2.03, 0.0, 2.07, -2.03, 1.72, 1.69, -2...."
2,Italy,Packaged Foods,EUR,Europe/Rome,MIL,Consumer Defensive,3831.0,8.794,5.971590e+07,68.513,...,2,9,3,28,37,11.23512,-6.57895,44.0,22.0,"[0.0, -3.33, -0.69, 4.17, -0.67, -0.67, -0.68,..."
3,United Kingdom,Other Precious Metals & Mining,EUR,Europe/Berlin,FRA,Basic Materials,108.0,1.402,2.123667e+08,81.212,...,0,1,4,3,40,25.00001,-21.52061,76.0,108.0,"[0.0, -0.13, -2.28, -1.17, -6.44, -1.54, -1.57..."
4,United States,Security & Protection Services,EUR,Europe/Berlin,FRA,Industrials,0.0,14.054,5.451780e+09,71.777,...,4,4,2,14,15,9.97164,-10.17502,21.0,260.0,"[0.0, -0.67, -0.68, 1.36, 0.0, -0.67, -0.68, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982,Switzerland,Specialty Industrial Machinery,CHF,Europe/Zurich,EBS,Industrials,1124.0,15.530,1.979479e+09,15.043,...,4,4,2,21,13,17.59738,-12.93413,59.0,259.0,"[0.0, -3.21, 0.83, -0.41, 0.62, -0.72, -0.72, ..."
7983,Japan,Business Equipment & Supplies,JPY,Europe/London,LSE,Industrials,1940.0,5.531,5.086319e+09,35.268,...,4,9,3,9,39,13.67696,-9.63956,290.0,64.0,"[0.0, 0.48, 0.0, -4.75, 0.0, 0.0, 0.0, 2.97, 0..."
7984,Japan,Electronic Gaming & Multimedia,JPY,Europe/London,LSE,Communication Services,3530.0,7.976,6.297095e+09,18.311,...,6,9,3,23,39,9.76675,-8.57953,197.0,48.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.51, 0.0..."
7985,Japan,Electronic Components,JPY,Europe/London,LSE,Technology,350.0,7.200,1.774490e+09,1.943,...,4,9,3,25,39,14.62789,-10.78806,65.0,137.0,"[0.0, 0.0, 0.0, 0.0, 2.95, 0.0, 0.0, 0.0, 0.0,..."


In [5]:
window_size = [1,5,10,20]

In [6]:
for w_size in window_size:
    stock_df=stock_df_backup.copy()
    column_names = list(stock_df.columns)
    beam_width = 20
    beam_depth = 3
    nr_bins = 8
    nr_saved = 50
    subgroup_size = len(stock_df) * 0.05
    target = 'growth_target'
    target_ind = column_names.index(target)
    stock_df['growth_target'] = stock_df['growth_target'].apply(lambda x: x[:len(x) - (len(x) % w_size)])
    stock_df_all['growth_target'] = stock_df_all['growth_target'].apply(lambda x: x[:len(x) - (len(x) % w_size)])
    print('length growth_target: ', len(stock_df.loc[0, 'growth_target']))
    data = stock_df.values.tolist()
    all_time_series = [i[target_ind] for i in data]
    all_time_series = np.array(all_time_series)
    targets_baseline = np.mean(all_time_series, axis=0)
    att_indices = list(range(0, len(column_names)))
    att_indices.remove(target_ind)
    att_columns = [column_names[i] for i in att_indices]
    types = bsm.categorize_columns_in_order(stock_df, att_columns)
    min_quality_improv = 0.1
    agg_func = np.mean
    results = bsm.beam_search_with_constraint(data, targets_baseline, column_names, beam_width, beam_depth, nr_bins, nr_saved,
                          subgroup_size, target, types, w_size, min_quality_improv, agg_func)
    q_res = []
    for res in results:
        q_res.append(res[0])
    
    column_count = {}
    
    # Iterating through each tuple in the data list
    for item in results:
        conditions = item[1]  # The second element contains the conditions (column names)
        for condition in conditions:
            column_name = condition[0]  # The first element in the condition tuple is the column name
            # Increment the count for this column name
            if column_name in column_count:
                column_count[column_name] += 1
            else:
                column_count[column_name] = 1
    column_count = dict(sorted(column_count.items(), key=lambda x: x[1], reverse=True))
    
    print('window size:',w_size,' quality mean:', np.mean(q_res), ' quality std:', np.std(q_res), ' column_count:', column_count)

length growth_target:  309
window size: 1  quality mean: 0.16811540389276836  quality std: 0.07054553979625233  column_count: {'growth_std_dev': 15, 'averageVolume10days': 7, 'growth_median': 6, 'growth_biggest_continuous_decrease_perc': 4, 'growth_biggest_continuous_increase_perc': 4, 'growth_range': 4, 'exchange': 4, 'growth_max': 3, 'growth_min': 3, 'sector': 3, 'currency': 2, 'country': 2, 'og_longest_continuous_increase': 1, 'exchangeTimezoneName': 1, 'marketCap': 1, 'og_longest_continuous_decrease': 1, 'og_trend_slope': 1}
length growth_target:  305
window size: 5  quality mean: 0.2395607031422335  quality std: 0.0972471419744955  column_count: {'growth_std_dev': 18, 'growth_biggest_continuous_decrease_perc': 6, 'growth_biggest_continuous_increase_perc': 5, 'growth_range': 5, 'growth_max': 5, 'marketCap': 5, 'sector': 4, 'growth_min': 3, 'og_window_min': 3, 'exchange': 3, 'country': 2, 'growth_mean': 2, 'og_trend_slope': 2, 'growth_median': 2, 'currency': 1, 'exchangeTimezoneName

In [7]:
window_size = [40, 60]
for w_size in window_size:
    stock_df = stock_df_backup.copy()
    column_names = list(stock_df.columns)
    beam_width = 20
    beam_depth = 3
    nr_bins = 8
    nr_saved = 50
    subgroup_size = len(stock_df) * 0.05
    target = 'growth_target'
    target_ind = column_names.index(target)
    stock_df['growth_target'] = stock_df['growth_target'].apply(lambda x: x[:len(x) - (len(x) % w_size)])
    stock_df_all['growth_target'] = stock_df_all['growth_target'].apply(lambda x: x[:len(x) - (len(x) % w_size)])
    print('length growth_target: ', len(stock_df.loc[0, 'growth_target']))
    data = stock_df.values.tolist()
    all_time_series = [i[target_ind] for i in data]
    all_time_series = np.array(all_time_series)
    targets_baseline = np.mean(all_time_series, axis=0)
    att_indices = list(range(0, len(column_names)))
    att_indices.remove(target_ind)
    att_columns = [column_names[i] for i in att_indices]
    types = bsm.categorize_columns_in_order(stock_df, att_columns)
    min_quality_improv = 0.1
    agg_func = np.mean
    results = bsm.beam_search_with_constraint(data, targets_baseline, column_names, beam_width, beam_depth, nr_bins,
                                              nr_saved,
                                              subgroup_size, target, types, w_size, min_quality_improv, agg_func)
    q_res = []
    for res in results:
        q_res.append(res[0])

    column_count = {}

    # Iterating through each tuple in the data list
    for item in results:
        conditions = item[1]  # The second element contains the conditions (column names)
        for condition in conditions:
            column_name = condition[0]  # The first element in the condition tuple is the column name
            # Increment the count for this column name
            if column_name in column_count:
                column_count[column_name] += 1
            else:
                column_count[column_name] = 1
    column_count = dict(sorted(column_count.items(), key=lambda x: x[1], reverse=True))

    print('window size:', w_size, ' quality mean:', np.mean(q_res), ' quality std:', np.std(q_res), ' column_count:',
          column_count)

length growth_target:  280
window size: 40  quality mean: 0.502637338290641  quality std: 0.15839708351196305  column_count: {'og_autocorr_lag1': 19, 'og_window_max': 12, 'og_window_min': 11, 'growth_mean': 9, 'growth_best_quarter': 8, 'growth_best_month': 7, 'growth_best_week_of_month': 5, 'og_trend_slope': 5, 'growth_std_dev': 4, 'og_mean': 2, 'growth_trend_slope': 2, 'enterpriseToEbitda': 1, 'growth_range': 1, 'og_std_dev': 1}
length growth_target:  300
window size: 60  quality mean: 0.7185209048412156  quality std: 0.20265130396872222  column_count: {'og_window_max': 19, 'growth_mean': 18, 'og_autocorr_lag1': 18, 'og_window_min': 13, 'growth_std_dev': 6, 'growth_best_quarter': 4, 'growth_best_month': 4, 'growth_min': 3, 'growth_trend_slope': 3, 'marketCap': 3, 'growth_range': 2, 'og_mean': 2, 'og_trend_slope': 2, 'og_std_dev': 1, 'growth_best_week_of_month': 1, 'sector': 1, 'og_median': 1}
